In [ ]:
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({'font.size': 16})

import pandas as pd
import numpy as np

In [ ]:
field_name = 'W3'

filename = f'/home/alberto/almacen/PAUS_data/{field_name}_pointings.csv'
point_info = pd.read_csv(filename)

point_info.head()

In [ ]:
# Extract unique pointing identifiers
pointing_ids = np.unique(point_info['pointing_id'])
print(len(pointing_ids))

# Compute the mean RA, DEC of the dithering
mean_ra = np.empty_like(pointing_ids)
mean_dec = np.empty_like(pointing_ids)
for i, point_id in enumerate(pointing_ids):
    mean_ra[i] = np.average(point_info['ra'][point_info['pointing_id'] == point_id])
    mean_dec[i] = np.average(point_info['dec'][point_info['pointing_id'] == point_id])

In [ ]:
# Load the photometry catalog
from load_paus_cat import load_paus_cat

path_to_cat = [f'/home/alberto/almacen/PAUS_data/catalogs/PAUS_3arcsec_{field_name}_extinction_corrected.pq']
cat = load_paus_cat(path_to_cat)

mask_NB_number = (cat['NB_number'] > -1)
cat['flx'] = cat['flx'][:, mask_NB_number]
cat['err'] = cat['err'][:, mask_NB_number]
cat['NB_mask'] = cat['NB_mask'][:, mask_NB_number]
for key in cat.keys():
    if key in ['flx', 'err', 'NB_mask', 'area']:
        continue
    cat[key] = cat[key][mask_NB_number]

stack_nb_ids = np.arange(12, 16 + 1)
synth_BB_flx = np.average(cat['flx'][stack_nb_ids],
                          weights=cat['err'][stack_nb_ids] ** -2,
                          axis=0)

N_sources = len(cat['ref_id'])

In [ ]:
# Assign each source the closest pointing center
pointing_id_Arr = np.empty(N_sources)
for src in range(N_sources):
    if not src % 100000:
        print(f'{src} / {N_sources}')
    where_pointing_id_Arr = np.argmin(
        (cat['RA'][src] - mean_ra) ** 2
        + (cat['DEC'][src] - mean_dec) ** 2
    )
    pointing_id_Arr[src] = pointing_ids[where_pointing_id_Arr]

# Save the array
save_path = f'/home/alberto/almacen/PAUS_data/catalogs/pointing_ids_{field_name}'
np.save(save_path, pointing_id_Arr)

In [ ]:
fig, ax = plt.subplots()

for pid in np.unique(pointing_id_Arr):
    mask = pointing_id_Arr == pid
    ax.scatter(cat['RA'][mask], cat['DEC'][mask], s=1, alpha=0.1)

ax.scatter(mean_ra, mean_dec)
ax.set(xlabel='RA', ylabel='DEC')

plt.show()

In [ ]:
# from paus_utils import data_tab as fil_properties
# from paus_utils import w_central
# from jpasLAEs.utils import flux_to_mag

# to_plot = np.arange(40)

# fig, ax = plt.subplots(figsize=(6, 4))

# for fil_id in to_plot:
#     # SNR versus magnitude
#     sigma = 5
#     pointing_depth_list = []

#     snr = cat['flx'][fil_id] / cat['err'][fil_id]
#     mag = flux_to_mag(cat['flx'][fil_id], w_central[fil_id])

#     for point_id in pointing_ids:
#         mask = (pointing_id_Arr.astype(int) == point_id)
#         if sum(mask) == 0:
#             pointing_depth_list.append(0)
#             continue
        
#         depth = np.median(mag[mask & (np.abs(snr - 5) < 0.1)])
#         pointing_depth_list.append(depth)

#     ax.scatter(pointing_ids, pointing_depth_list,
#                label=fil_properties['name'][fil_id])

#     fil_properties['name']

# ax.set_ylim(20, 24)
# ax.legend(fontsize=10)

# plt.show()